# Big Data Analytics Project 

In [5]:
import pandas as pd

# Load the JSON file into a Pandas DataFrame
salesdf = pd.read_json('sales.json', lines=True)


In [6]:
!pip install pandas

In [7]:
num_rows, num_columns = salesdf.shape

print(f"Number of Rows: {num_rows}")
print(f"Number of Columns: {num_columns}")

Number of Rows: 5000
Number of Columns: 7


In [9]:
from IPython.display import display, HTML

# Assuming 'salesdf' is your DataFrame
html_table = salesdf.head(10).to_html(classes='customtable', index=False)

# Define custom CSS styles to color the header
styles = """
    <style>
        .customtable {
            border-collapse: collapse;
            width: 100%;
        }
        .customtable th {
            background-color: #4e20ce; /* background color for header */
            color: white; /* White text color for header */
            border: 1px solid black;
            padding: 8px;
            text-align: left;
        }
        .customtable td {
            border: 1px solid black;
            padding: 8px;
            text-align: left;
        }
    </style>
"""

# Display the HTML with custom styles
display(HTML(styles + html_table))


_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod
{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},"[{'name': 'printer paper', 'tags': ['office', 'stationary'], 'price': {'$numberDecimal': '40.01'}, 'quantity': 2}, {'name': 'notepad', 'tags': ['office', 'writing', 'school'], 'price': {'$numberDecimal': '35.29'}, 'quantity': 2}, {'name': 'pens', 'tags': ['writing', 'office', 'school', 'stationary'], 'price': {'$numberDecimal': '56.12'}, 'quantity': 5}, {'name': 'backpack', 'tags': ['school', 'travel', 'kids'], 'price': {'$numberDecimal': '77.71'}, 'quantity': 2}, {'name': 'notepad', 'tags': ['office', 'writing', 'school'], 'price': {'$numberDecimal': '18.47'}, 'quantity': 2}, {'name': 'envelopes', 'tags': ['stationary', 'office', 'general'], 'price': {'$numberDecimal': '19.95'}, 'quantity': 8}, {'name': 'envelopes', 'tags': ['stationary', 'office', 'general'], 'price': {'$numberDecimal': '8.08'}, 'quantity': 3}, {'name': 'binder', 'tags': ['school', 'general', 'organization'], 'price': {'$numberDecimal': '14.16'}, 'quantity': 3}]",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@witwuta.sv', 'satisfaction': 4}",True,Online
{'$oid': '5bd761dcae323e45a93ccfe9'},{'$date': '2015-08-25T10:01:02.918Z'},"[{'name': 'envelopes', 'tags': ['stationary', 'office', 'general'], 'price': {'$numberDecimal': '8.05'}, 'quantity': 10}, {'name': 'binder', 'tags': ['school', 'general', 'organization'], 'price': {'$numberDecimal': '28.31'}, 'quantity': 9}, {'name': 'notepad', 'tags': ['office', 'writing', 'school'], 'price': {'$numberDecimal': '20.95'}, 'quantity': 3}, {'name': 'laptop', 'tags': ['electronics', 'school', 'office'], 'price': {'$numberDecimal': '866.5'}, 'quantity': 4}, {'name': 'notepad', 'tags': ['office', 'writing', 'school'], 'price': {'$numberDecimal': '33.09'}, 'quantity': 4}, {'name': 'printer paper', 'tags': ['office', 'stationary'], 'price': {'$numberDecimal': '37.55'}, 'quantity': 1}, {'name': 'backpack', 'tags': ['school', 'travel', 'kids'], 'price': {'$numberDecimal': '83.28'}, 'quantity': 2}, {'name': 'pens', 'tags': ['writing', 'office', 'school', 'stationary'], 'price': {'$numberDecimal': '42.9'}, 'quantity': 4}, {'name': 'envelopes', 'tags': ['stationary', 'office', 'general'], 'price': {'$numberDecimal': '16.68'}, 'quantity': 2}]",Seattle,"{'gender': 'M', 'age': 50, 'email': 'keecade@hem.uy', 'satisfaction': 5}",False,Phone
{'$oid': '5bd761dcae323e45a93ccfea'},{'$date': '2017-06-22T09:54:14.185Z'},"[{'name': 'notepad', 'tags': ['office', 'writing', 'school'], 'price': {'$numberDecimal': '30.08'}, 'quantity': 1}, {'name': 'binder', 'tags': ['school', 'general', 'organization'], 'price': {'$numberDecimal': '24.68'}, 'quantity': 7}, {'name': 'notepad', 'tags': ['office', 'writing', 'school'], 'price': {'$numberDecimal': '32.1'}, 'quantity': 4}, {'name': 'pens', 'tags': ['writing', 'office', 'school', 'stationary'], 'price': {'$numberDecimal': '31.41'}, 'quantity': 3}, {'name': 'printer paper', 'tags': ['office', 'stationary'], 'price': {'$numberDecimal': '43.56'}, 'quantity': 3}, {'name': 'pens', 'tags': ['writing', 'office', 'school', 'stationary'], 'price': {'$numberDecimal': '68.95'}, 'quantity': 4}, {'name': 'notepad', 'tags': ['office', 'writing', 'school'], 'price': {'$numberDecimal': '8.57'}, 'quantity': 4}, {'name': 'backpack', 'tags': ['school', 'travel', 'kids'], 'price': {'$numberDecimal': '106.6'}, 'quantity': 2}, {'name': 'laptop', 'tags': ['electronics', 'school', 'office'], 'price': {'$numberDecimal': '596.33'}, 'quantity': 4}]",Denver,"{'gender': 'M', 'age': 51, 'email': 'worbiduh@vowbu.cg', 'satisfaction': 5}",False,In store
{'$oid': '5bd761dcae323e45a93ccfeb'},{'$date': '2015-02-23T09:53:59.343Z'},"[{'name': 'binder', 'tags': ['school', 'general', 'organization'], 'price': {'$numberDecimal': '20.08'}, 'quantity': 1}, {'name': 'pens', 'tags': ['writing', 'office', 'school', 'stationary'], 'price': {'$numberDecimal': '23.08'}, 'quantity': 4}, {'name': 'backpac